In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

# 하이퍼파라미터 설정
batch_size = 64
epochs = 50
input_height = 128
input_width = 128
input_channels = 1   # 흑백 이미지
num_classes = 4

# 데이터 디렉토리 경로
data_dir = '/content/BrainMRI'
assert os.path.exists(data_dir), "데이터 폴더가 존재하지 않습니다."


# 1. 데이터 증강 및 로딩


# ImageDataGenerator: 학습용은 증강 포함, 검증용은 증강 제외 및 정규화만
train_datagen = ImageDataGenerator(
    rescale=1./255,                   # 0~1 정규화
    rotation_range=30,                # -30 ~ +30도 회전
    zoom_range=0.2,                  # 80%~120% 확대/축소
    width_shift_range=0.2,           # 좌우 20% 이동
    height_shift_range=0.2,          # 상하 20% 이동
    horizontal_flip=True,            # 좌우 반전
    validation_split=0.3             # 30%는 검증용으로 분리
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

# 학습용 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(input_height, input_width),
    color_mode='grayscale',     # 흑백
    batch_size=batch_size,
    class_mode='categorical',   # 다중 클래스 분류용 one-hot 인코딩
    subset='training',
    shuffle=True
)

# 검증용 데이터 생성기
val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=(input_height, input_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


# 2. 모델 구성 (GRU)


# GRU 입력은 (batch, timesteps=128, features=128)
# 이미지 (128,128,1) → (128,128)로 reshape하여 시퀀스처럼 처리

inputs = layers.Input(shape=(input_height, input_width, input_channels))

# 흑백 채널 제거 후 시퀀스 형태로 변환
x = layers.Reshape((input_height, input_width))(inputs)  # (batch, 128, 128)

# GRU 레이어 3층 쌓기 (마지막 레이어는 return_sequences=False)
x = layers.GRU(256, return_sequences=True)(x)
x = layers.GRU(256, return_sequences=True)(x)
x = layers.GRU(256)(x)  # 마지막 타임스텝 출력만 사용

# 출력층: 클래스 수만큼 노드, softmax 활성화 함수
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


# 3. 콜백 정의


earlystop = callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True, verbose=1
)

checkpoint = callbacks.ModelCheckpoint(
    'best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',          # 검증 손실 기준 학습률 감소
    factor=0.5,                 # 학습률 50% 감소
    patience=2,                 # 2회 연속 개선 없으면 감소
    verbose=1
)


# 4. 모델 학습


history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[earlystop, checkpoint, reduce_lr]
)

# 5. 학습 결과 시각화


plt.figure(figsize=(12, 5))

# 손실 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
